In [1]:
'''
В данной версии КП будет применяться замена категориальных переменных dummy-переменными и замена nan-значений значениями
из предыдущих строк вместо медианных значений.
'''

import numpy as np, pandas as pd, os
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor as RFReg


wrk='C:\\Users\\Lenovo\\Documents\\GeekBrains\\Python DataScience Kurs'
train=pd.read_csv(os.path.join(wrk,'train.csv'))

train=train.drop(['Id'],axis=1) #удаляем стоблбец id - он нам не нужен
cols=train.columns.to_numpy()

res={}

for x in cols:
    res[x]=train[x].value_counts().sort_index()

In [2]:
'''
Результаты анализа значений:
    
    DistrictId - 205 возможных значений. Прямой/обратной зависимости (чем больше id, тем лучше/хуже район)
            стоимости квартиры от данного параметра не предполагается. Поэтому отнесем к дискретным;
    Rooms - 9 различных значений. Предполагается зависимость цены квартиры от кол-ва комнат, поэтому отнесем этот
            параметр к непрерывным;
    Square - 10000 различных значений. ОТносим к непрерывным признакам;
    LifeSquare - 7887 различных значений. Относим к непрерывным значениям; Значения nan будем заменять значениями
            Square-KitchenSquare ПОСЛЕ редактирования KitchenSquare;
    KitchenSquare - 58 различных значений. Предполагается зависимость цены квартиры от площади кухни, поэтому отнесем этот
            параметр к непрерывным. Почти 25% значений равны 1 => заменим их на медиану без учета 1.
            Есть ошибочные значения 1970 и 2014 (видимо, год постройки дома);
    Floor - 33 различных значения. Предполагается зависимость цены квартиры от ее этажа, поэтому отнесем этот
            параметр к непрерывным;
    HouseFloor - 44 различных значения. Предполагается зависимость цены квартиры от этажности дома, поэтому
            отнесем этот параметр к непрерывным; ОБРАБОТКА ОШИБОЧНЫХ ЗНАЧЕНИЙ: Floor <= HouseFloor и HouseFloor!=0;
    HouseYear - 97 различных значений. Предполагается зависимость цены квартиры от года постройки дома, поэтому
            отнесем этот параметр к непрерывным; Ошибочные значения: 4968 и 20052011;
    Ecology_1 - 129 непрерывных значений. Отнесем этот параметр к непрерывным;
    Ecology_2 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    Ecology_3 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    Social_1 - 51 различное значение. Предполагается зависимость цены квартиры от социальных условий, поэтому
            отнесем этот параметр к непрерывным;
    Social_2 - 142 различных значения. Аналогично;
    Social_3 - 30 различных значений. Аналогично;
    Healthcare_1 - 79 различных значений. Аналогично;
    Helthcare_2 - 7 различных значений. Из-за этого отнесем этот параметр к дискретным;
    Shops_1 - 16 различных значений. Предполагается зависимость цены квартиры от доступности магазинов, поэтому
            отнесем этот параметр к непрерывным;
    Shops_2 - 2 значения (A/B). Отнесем этот параметр к дискретным;
    
'''

#заполняем nan-значения предыдущими
train=train.fillna(method='pad')
train['Healthcare_1'][0]=train['Healthcare_1'].median() #в первой строке осталось nan-значение

#переводим категориальные переменные в dummy-переменные
train=pd.get_dummies(train,prefix=['eco2','eco3','sh2'])
cols=train.columns.to_numpy()
cols_x=list(cols)
cols_x.remove('Price')


<ipython-input-2-fa6ed40f0bd7>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Healthcare_1'][0]=train['Healthcare_1'].median() #в первой строке осталось nan-значение


In [3]:
'''определление важности признаков'''
feats_tab=train.copy()
answ_tab=feats_tab['Price']
feats_tab=feats_tab.drop(['Price'],axis=1)

feats_tab=feats_tab.to_numpy()
answ_tab=answ_tab.to_numpy()

reg=DecisionTreeRegressor(criterion='mae')
reg.fit(feats_tab,answ_tab)
feat_importance=reg.tree_.compute_feature_importances(normalize=True)
feat_importance=np.vstack((cols_x,feat_importance))
feat_importance=np.transpose(feat_importance)
feat_importance=pd.DataFrame(feat_importance)
feat_importance.columns=['Param','Importance']
feat_importance['Importance']=pd.to_numeric(feat_importance['Importance'])
feat_importance=feat_importance.sort_values(by='Importance',ascending=False)
feat_importance=feat_importance.reset_index().drop(['index'],axis=1) #сброс индекса
print(feat_importance)

print('Сумма значимости первых 5 признаков: ',str(sum(feat_importance['Importance'][:5])))
print('Сумма значимости первых 10 признаков: ',str(sum(feat_importance['Importance'][:10])))

feats_top10=feat_importance.loc[:9]
print('10 наиболее значимых признаков:\n',feats_top10)

            Param  Importance
0           Rooms    0.187289
1          Square    0.151833
2   KitchenSquare    0.103158
3        Social_1    0.067442
4        Social_3    0.061059
5       HouseYear    0.055267
6      LifeSquare    0.055035
7        Social_2    0.052478
8      DistrictId    0.049232
9       Ecology_1    0.046570
10          Floor    0.045152
11     HouseFloor    0.039932
12   Healthcare_1    0.037142
13        Shops_1    0.031465
14    Helthcare_2    0.011580
15          sh2_A    0.001823
16         eco3_B    0.001620
17          sh2_B    0.000856
18         eco3_A    0.000600
19         eco2_B    0.000254
20         eco2_A    0.000214
Сумма значимости первых 5 признаков:  0.570780626947613
Сумма значимости первых 10 признаков:  0.8293627965959468
10 наиболее значимых признаков:
            Param  Importance
0          Rooms    0.187289
1         Square    0.151833
2  KitchenSquare    0.103158
3       Social_1    0.067442
4       Social_3    0.061059
5      HouseYear   

In [5]:
'''
Таким образом, в первую очередь стоимость квартиры зависит от площади и кол-ва комнат, в меньшей степени
влияют площадь кухни, социальные условия, жилая площадь, район, год постройки дома и этаж квартиры.

В наименьшей степени на стоимость квартиры влияют близость к магазинам, торговым центрам,
учреждениям здравоохранения, а также экологические показатели;

Если проводить простую замену nan предыдущими значениями, то информативность первых 5/10 признаков падает:
0.58/0.83 против 0.60/0.86
'''

#подготовка тестовой выборки к разделению для обучения множества регрессоров
X=train.copy()
y=X['Price']
X=X.drop(['Price'],axis=1)
X=X.to_numpy()
y=y.to_numpy()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)

#подготовка модели и настройка алгоритма перебора
reg=DecisionTreeRegressor(criterion='mae')
params={'max_depth': np.arange(5,15)} #сначала перебор по глубине дерева
GS=GridSearchCV(reg, params)
GS.fit(X_train,y_train)

test_react=GS.best_estimator_.predict(X_test)
r2_1=r2_score(y_test,test_react)
print(r2_1,GS.best_params_)

0.6151389381787107 {'max_depth': 9}


In [6]:
'''Наибольшее значение r2 показало дерево, имеющее максимальную глубину 9. Оно равно 0.6151 (+0.0104).
   Поэтому проведем еще один поиск, на этот раз по min_samples_leaf. Макс. глубину дерева
   зафиксируем на 9'''
reg2=DecisionTreeRegressor(criterion='mae',max_depth=9)
params2={'min_samples_leaf': np.arange(2,10)} #затем перебор по мнинмальному кол-ву объектов в узле
GS2=GridSearchCV(reg2, params2)
GS2.fit(X_train,y_train)

test_react=GS2.best_estimator_.predict(X_test)
r2_2=r2_score(y_test,test_react)
print(r2_2,GS2.best_params_)

0.6467586340125306 {'min_samples_leaf': 5}


In [7]:
'''Наибольшее значение r2 показало дерево, имеющее минимальное кол-во
   объектов в узле, равное 5, при макс. глубине дерева в 9. Значение r2 немного улучшилось и стало
   равно 0.6468 (+0.180).
   
   Создадим ансамбли из деревьев с подобными параметрами и обучим его. Кол-ва деревьев выберем
   равными 5, 10, 20, 30, 50, 100, 200, 500, 1000.
   
   ОБучим модель с полученными параметрами уже на всей обучающей выборке'''

min_objcts=5

ens1=RFReg(n_estimators=5,max_depth=8,min_samples_leaf=min_objcts)
ens1.fit(X_train,y_train)
test_react=ens1.predict(X_test)
r2_ens1=r2_score(y_test,test_react)

ens2=RFReg(n_estimators=10,max_depth=8,min_samples_leaf=min_objcts)
ens2.fit(X_train,y_train)
test_react=ens2.predict(X_test)
r2_ens2=r2_score(y_test,test_react)

ens3=RFReg(n_estimators=20,max_depth=8,min_samples_leaf=min_objcts)
ens3.fit(X_train,y_train)
test_react=ens3.predict(X_test)
r2_ens3=r2_score(y_test,test_react)

ens4=RFReg(n_estimators=30,max_depth=8,min_samples_leaf=min_objcts)
ens4.fit(X_train,y_train)
test_react=ens4.predict(X_test)
r2_ens4=r2_score(y_test,test_react)

ens5=RFReg(n_estimators=50,max_depth=8,min_samples_leaf=min_objcts)
ens5.fit(X_train,y_train)
test_react=ens5.predict(X_test)
r2_ens5=r2_score(y_test,test_react)

ens6=RFReg(n_estimators=100,max_depth=8,min_samples_leaf=min_objcts)
ens6.fit(X_train,y_train)
test_react=ens6.predict(X_test)
r2_ens6=r2_score(y_test,test_react)

ens7=RFReg(n_estimators=200,max_depth=8,min_samples_leaf=min_objcts)
ens7.fit(X_train,y_train)
test_react=ens7.predict(X_test)
r2_ens7=r2_score(y_test,test_react)

ens8=RFReg(n_estimators=500,max_depth=8,min_samples_leaf=min_objcts)
ens8.fit(X_train,y_train)
test_react=ens8.predict(X_test)
r2_ens8=r2_score(y_test,test_react)

ens9=RFReg(n_estimators=1000,max_depth=8,min_samples_leaf=min_objcts)
ens9.fit(X_train,y_train)
test_react=ens9.predict(X_test)
r2_ens9=r2_score(y_test,test_react)

print('5 - {:.4f}\n10 - {:.4f}\n20 - {:.4f}\n30 - {:.4f}\n50 - {:.4f}\n100 - {:.4f}\n200 - {:.4f}\n500 - {:.4f}\n1000 - {:.4f}\n'\
      .format(r2_ens1,r2_ens2,r2_ens3,r2_ens4,r2_ens5,r2_ens6,r2_ens7,r2_ens8,r2_ens9))

5 - 0.6790
10 - 0.6829
20 - 0.6876
30 - 0.6877
50 - 0.6885
100 - 0.6902
200 - 0.6910
500 - 0.6910
1000 - 0.6910



In [ ]:
'''
Наилучший результат показали ансамбли из 200-1000 деревьев (r2=0.6910).
Результат остался практически прежним, следовательно, смена способа обработки данных практически не повлияла на результат,
что свидетельствует об одинаковом качестве обеих предобработок.

Так как результат практически не изменился, то дальнейшее обучение модели и вычисление цены на тестовой выборке проводить
не будем.
'''